# Carga y preprocesamiento de datos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Carga
df = pd.read_csv('data/departamentos.csv')
# Conversión del tipo de dato
df['precio'] = df['precio'].astype(float)
df['gastos_comunes'] = df['gastos_comunes'].astype(float)
df['superficie_total'] = df['superficie_total'].replace(to_replace=",", value='.', regex=True)
df['superficie_total'] = df['superficie_total'].astype(float)
df['superficie_util'] = df['superficie_util'].replace(to_replace=",", value='.', regex=True)
df['superficie_util'] = df['superficie_util'].astype(float)
# Filtro de variables no consideradas
df = df[df.columns[~df.columns.isin(['titulo','currency_symbol','search_gc', 'descripcion','ubicacion','url'])]]
# Precio total
df['total'] = df['precio'] + df['gastos_comunes']
df = df[df.columns[~df.columns.isin(['precio','gastos_comunes','superficie_util'])]]
# Filtrar departamentos menores o iguales a 600.000 CLP
df = df.loc[(df['total'] <= 600000)]
# Forma del dataset
df.shape

(267, 10)

## Revisión de NaNs y su tratamiento

In [3]:
# Revisión de NaN
pd.isna(df).sum()

preferencia             0
comuna                  0
estacion_cercana       59
distancia_estacion     59
dormitorios             1
baños                   1
estacionamientos       95
bodegas               171
superficie_total        4
total                   0
dtype: int64

#### Tratamiento de variables "estación_cercana" y "distancia_estacion"

Si el valor en "estación_cercana" y "distancia_estacion" no esta presente, es por que el departamento no presenta estaciones cercanas

In [4]:
df['estacion_cercana'] = df['estacion_cercana'].fillna('Sin estación cercana')
# Si la estación esta lejana, se añade 3000 mts como referencia
# Recordar que una estación cercama se considera como menor a 2000 mts 
df['distancia_estacion'] = df['distancia_estacion'].fillna(3000)

#### Tratamiento de variables "estacionamientos" y "bodegas"

Se considera que si el valor de la "estacionamientos" y "bodegas" no estan presentes, es por que el departamento no las posee, por lo tanto se añade 0

In [5]:
df['estacionamientos'] = df['estacionamientos'].fillna(0)
df['bodegas'] = df['bodegas'].fillna(0)

Las otras variables faltantes representan un subconjuntos marginal respecto al total, por lo que se decide remover del dataset

In [6]:
df = df.dropna()
df.shape

(261, 10)

# Análisis Exploratorio de Datos (AED)

In [7]:
# Observación de los primeros datos del conjunto
df.head()

,preferencia,comuna,estacion_cercana,distancia_estacion,dormitorios,baños,estacionamientos,bodegas,superficie_total,total
2,0,La Reina,Fernando Castillo Velasco,1433.0,2.0,1.0,0.0,0.0,45.0,375000.0
9,0,La Reina,Simón Bolívar,485.0,1.0,1.0,1.0,0.0,24.0,380000.0
25,0,La Florida,Rojas Magallanes,462.0,2.0,2.0,1.0,1.0,65.0,540000.0
27,0,La Florida,Vicente Valdés,70.0,2.0,1.0,1.0,0.0,58.0,505000.0
28,0,La Florida,Los Quillayes,835.0,3.0,1.0,0.0,0.0,55.0,405000.0


In [8]:
# Tipos de datos del conjunto
df.dtypes

preferencia             int64
comuna                 object
estacion_cercana       object
distancia_estacion    float64
dormitorios           float64
baños                 float64
estacionamientos      float64
bodegas               float64
superficie_total      float64
total                 float64
dtype: object

# Metodología de clasificación

In [9]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn import svm

In [10]:
# Para usar los métodos SVM las variables categoricas "comuna" y "estacion_cercana" se transforman a variables dummys
df = pd.get_dummies(df)

In [11]:
# Separación de etiqueta Y y caracteristicas X
y = df.iloc[:,0]
X = df.iloc[:,1:]
# Subconjunto de entrenamiento y testing de 80 y 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((195, 58), (195,), (66, 58), (66,))

## Modelo SVM

Prueba del modelo SVM por defecto: C=1, kernel='rbf', degree=3

In [12]:
# Definición del modelo SVM
clf = svm.SVC()
# Entrenamiento del modelo
clf.fit(X_train, y_train)
# Precisión
clf.score(X_test, y_test)
# Predicción de X test
y_test_pred = clf.predict(X_test)

#### Cross Validation

In [13]:
# Clasificación SVC por defecto
accuracy = cross_val_score(clf, X, y, cv=5)
print(accuracy)
# Precisión promedio
print("Precisión promedio: ", accuracy.mean() * 100)

[0.83018868 0.84615385 0.84615385 0.84615385 0.84615385]
Precisión promedio:  84.29608127721335


Podemos observar que la precisión del modelo a través de *cross-validation* con k=10 es en promedio de 84.6%

In [14]:
# Obtención de predicciones por medio de validación cruzada
y_pred = cross_val_predict(clf, X, y, cv=10)
# Matriz de confusión
confusion_matrix(y, y_pred)

array([[220,   0],
       [ 41,   0]])

#### f1 score

In [15]:
f1_score(y_test, y_test_pred)

0.0

Si bien se observa la matriz de confusión puede clasificar perfectamente las preferencias 0, ha fallado el 100% de las preferencias 1, y tal como se conoce la naturaleza del problema, es más interesante poder clasificar correctamente la clase 1

#### auc score

In [16]:
roc_auc_score(y_test, y_test_pred)

0.5

## Prueba automatica de diferentes modelos SVM

In [17]:
for kernel in ['linear', 'poly', 'rbf', 'sigmoid'] :
    print('====> Kernel : ', kernel)
    for C in [0.01, 0.1, 1, 1.5, 3, 5, 10, 20, 50, 100] :
        print('==> C : ', C)
        clf_iter = svm.SVC(kernel=kernel, C=C)
        accuracy = cross_val_score(clf_iter, X, y, cv=5)
        print("Precisión promedio: ", accuracy.mean() * 100)
        clf_iter.fit(X_train, y_train)
        y_test_pred = clf_iter.predict(X_test)
        f1 = f1_score(y_test, y_test_pred, average='macro')
        confusion_matrix(y_test, y_test_pred)
        print('f1 : ', f1)
        aucscore = roc_auc_score(y_test, y_test_pred, average='macro')
        print('auc : ', aucscore)

====> Kernel :  linear
==> C :  0.01
Precisión promedio:  83.53410740203194
f1 :  0.46341463414634143
auc :  0.5
==> C :  0.1
Precisión promedio:  82.38026124818578
f1 :  0.4590163934426229
auc :  0.49122807017543857
==> C :  1
Precisión promedio:  83.53410740203194
f1 :  0.4590163934426229
auc :  0.49122807017543857
==> C :  1.5
Precisión promedio:  83.14949201741655
f1 :  0.4590163934426229
auc :  0.49122807017543857
==> C :  3
Precisión promedio:  82.76487663280116
f1 :  0.4590163934426229
auc :  0.49122807017543857
==> C :  5
Precisión promedio:  82.38026124818578
f1 :  0.4590163934426229
auc :  0.49122807017543857
==> C :  10
Precisión promedio:  83.14949201741655
f1 :  0.4590163934426229
auc :  0.49122807017543857
==> C :  20
Precisión promedio:  82.38026124818578
f1 :  0.46341463414634143
auc :  0.5
==> C :  50


# Conclusión

Los métodos SVM entrenados no ha aportado buenos resultados en general para clasificar, el mejor modelo SVM es uno entrenado con kernel = lineal y C = 1 obtuvo el mejor desempeño en relación a los otros modelos entrenados, sin embargo, el rendimiento sigue siendo muy pobre para considerarse útil en su utilización. Es posible intentar mejorar el modelo aumentando la cantidad de datos de entrenamiento, o añadir más caracteristicas predictoras de departamentos o de mejor calidad explicativa. También es posible revisar más combinaciones de kernel, parametros C y otros parametros de SVM en el modelo para encontrar uno que aporte un rendimiento mucho mejor a la hora de clasificar, además de probar otros diferentes métodos de clasificación como bayes ingenuo o arboles de decisión.